## This Notebook trains various LGBM Models for each of the six outcomes.
## It also trains models with both class weights, and with no class weights

In [1]:
import  numpy as np
import pandas as pd
import statsmodels.api as sm

from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV

from sklearn.preprocessing import QuantileTransformer, quantile_transform

from sklearn.metrics import median_absolute_error, r2_score, mean_squared_error
# ....

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
# import lightgbm as lgb 

from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.pipeline import Pipeline, make_pipeline

from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from sklearn import preprocessing 
import pickle
import joblib



%matplotlib inline

In [2]:
# Load cleaned data
with open('cleaned_data_label_encoding.pickle', 'rb') as handle:
    background_imputed_tot = pickle.load(handle)
    
with open('X_train_label_encoding.pickle', 'rb') as handle:
    X_train = pickle.load(handle)

with open('X_CV_label_encoding.pickle', 'rb') as handle:
    X_CV = pickle.load(handle)

with open('x_test_label_encoding.pickle', 'rb') as handle:
    x_test = pickle.load(handle)
    
with open('x_leaderboard_label_encoding.pickle', 'rb') as handle:
    x_leaderboard = pickle.load(handle)
    
with open('y_train_label_encoding.pickle', 'rb') as handle:
    y_train = pickle.load(handle)
    
with open('y_CV_label_encoding.pickle', 'rb') as handle:
    y_CV = pickle.load(handle)
    
background = pd.read_csv('FFChallenge_v5/background.csv', sep=',', header=0,index_col=0,low_memory=False)
# train.csv contains 2,121 rows (one per child in the training set) and 7 columns.
train = pd.read_csv('FFChallenge_v5/train.csv', sep=',', header=0, index_col=0,low_memory=False)
########### Holdout dataset for internal testing only
test = pd.read_csv('test.csv',header=0, index_col=0,low_memory=False)
leaderboard = pd.read_csv('leaderboard.csv', header=0, index_col=0,low_memory=False)
leaderboard = leaderboard.dropna()

In [4]:
# Final stats and useful variables associated with each column

numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v in [np.float,np.int,np.int64]]
non_numerical_columns = [c for c,v in background_imputed_tot.dtypes.iteritems() if v not in [np.float,np.int,np.int64]]


# The categorical columns are the intersection of both the non_numerical columns and the ones that have less than 15 distinct levels
# categorical_cols_lst = list(set(categorical_bools.columns).union(set(non_numerical_columns)))

continuous_cols_lst = list()
continuous_cols_lst = background_imputed_tot.T.loc[(background_imputed_tot.apply(pd.Series.nunique) >= 15).values==True].index.to_list()

non_continuous_lst = set(background_imputed_tot.columns.tolist())-set(continuous_cols_lst)


print('Number of continuous columns %s' % len(continuous_cols_lst))
# background = background[numerical_columns]
# background.head()

Number of continuous columns 601


### The Following cell trains three LGBM models: GBDT, DART and GOSS using no class weights. The models trained with class weights are provided in cells below this one
### The hyperparameters of each model are tuned using 20 iterations of bayesian optimization, with repeated cross validation and repeated stratified cross validation with 3 folds and 2 repeats.
### Early stopping is used to help minimize overfitting

In [5]:
from datetime import datetime 
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from sklearn.model_selection import RepeatedKFold,RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization


variables = test.columns.to_list()


prediction = pd.read_csv('FFChallenge_v5/prediction.csv', sep=',', header=0, index_col=0)

best_params_lgbm = dict()


# Bayesian optimization function for regression-based models
def optimize_gbm_regressor(data, targets,boosting_type):
    """Apply Bayesian Optimization to GBM parameters."""

    def gbm_crossval(num_leaves, max_depth, learning_rate,
                     n_estimators,num_iteration,subsample,min_child_samples,
                    colsample_bytree,reg_alpha,reg_lambda,):
        """GB cross validation.
        """
        # Parameters to be optimized
        clf = LGBMRegressor(
            num_leaves=int(num_leaves),
            max_depth=int(max_depth),
            learning_rate=learning_rate,
            n_estimators=int(n_estimators),
            num_iteration=int(num_iteration),
            boosting_type = boosting_type,
            subsample = subsample,
            min_child_samples = int(min_child_samples),
            colsample_bytree=colsample_bytree,
            reg_alpha = reg_alpha,
            reg_lambda = reg_lambda,
        )
        cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=1)
        cval = cross_val_score(clf, data, targets, scoring='neg_mean_squared_error', cv=cv,n_jobs=-1)
        print(cval)
        return cval.mean()
    
    # Parameters to be optimized
    optimizer = BayesianOptimization(
        f=gbm_crossval,
        pbounds={
            'num_leaves': (2, 30),
            'max_depth': (1, 15),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (10,1000),
            'num_iteration':(50,200),
            'subsample': (0.5,1.0),
            'min_child_samples' : (10, 200),
            'colsample_bytree':(0.5,1),
            'reg_alpha': (0.0, 1.0),
            'reg_lambda': (0.0, 1.0),
        },
        random_state=12345,
        verbose=3
    )
    optimizer.maximize(n_iter=15, init_points=5)

    print("Final result:", optimizer.max)
    return optimizer.max


# Bayesian optimization function for classification-based models
def optimize_gbm_classifier(data, targets,boosting_type):
    """Apply Bayesian Optimization to GBM parameters."""
#     self.boosting_type = boosting_type

    def gbm_crossval(num_leaves, max_depth, learning_rate,
                     n_estimators,num_iteration,subsample,min_child_samples,
                    colsample_bytree,reg_alpha,reg_lambda,):
        """GB cross validation.
        """
        clf = LGBMClassifier(
        num_leaves=int(num_leaves),
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        num_iteration=int(num_iteration),
        boosting_type = boosting_type,
        subsample = subsample,
        min_child_samples = int(min_child_samples),
        colsample_bytree=colsample_bytree,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        )
        cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=123)
        cval = cross_val_score(clf, data, targets, scoring='neg_brier_score', cv=cv,n_jobs=-1)
#         print(cval)
        return cval.mean()
    
    # Parameters to be optimized
    optimizer = BayesianOptimization(
        f=gbm_crossval,
        pbounds={
            'num_leaves': (2, 30),
            'max_depth': (1, 15),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (10,1000),
            'num_iteration':(50,200),
            'subsample': (0.5,1.0),
            'min_child_samples' : (10, 200),
            'colsample_bytree':(0.5,1),
            'reg_alpha': (0.0, 1.0),
            'reg_lambda': (0.0, 1.0),
        },
        random_state=12345,
        verbose=3
    )
    optimizer.maximize(n_iter=15, init_points=5)

    print("Final result:", optimizer.max)
    return optimizer.max


classification_vars = ['eviction','layoff','jobTraining']




boosting_types = ['gbdt','dart','goss'] # List of the three boosting types

best_params_each_model = dict()

accuracies_each_model = dict()

timings_each_model = dict()
for boosting_type in boosting_types: # Go through all three of the different boosting types
    print('------------- ',boosting_type, ' --------------')
    print('\n')
    lgbm_accuracies = dict()
    lgbm_timings = dict()
    for variable in variables: # Go through all the variable outcomes
#     for variable in ['grit','layoff']:
    # variable = 'eviction'
        print(variable)
        start=datetime.now()
        # Let's drop instances where the response is NA
        y_train_no_na = y_train[variable].dropna()
        X_train_no_na = X_train.loc[y_train_no_na.index.values]

        y_CV_no_na = y_CV[variable].dropna()
        X_CV_no_na = X_CV.loc[y_CV_no_na.index.values]

        mask = leaderboard[variable].isna()
        y_leaderboard = leaderboard[variable]

        #training our model using light gbm

        # specify parameters and distributions to sample from

        # If the outcomes are continuous use the regressor. Otherwise use the classifier
        if variable not in classification_vars:
            print(variable)
            # Run bayesian optimization to get optimal hyperparameters
            params = optimize_gbm_regressor(X_train_no_na, y_train_no_na,boosting_type)
            best_params = params['params']  
            print(1)

            # train a new model using the best hyperparameters 
            lgbm=LGBMRegressor(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                   n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                        num_iteration = int(best_params['num_iteration']),
                        boosting_type=boosting_type, 
                        subsample = best_params['subsample'],
                        min_child_samples = int(best_params['min_child_samples']),
                        colsample_bytree=best_params['colsample_bytree'],
                        reg_alpha = best_params['reg_alpha'],
                        reg_lambda = best_params['reg_lambda'],
                               silent = True)

            # I used early stopping to help prevent overfitting the model on the training set (stops the model once it's performance doesn't
            # improve for 5 iterations on the validation set)
            lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2',
             early_stopping_rounds = 5)

            
            # For early stopping (i.e. early stopping at 10 rounds with no improvement to cv)
            # The model that displayed the score 10 rounds ago does not exist anymore (a few trees have been dropped).
            # To get the best score you got 20 rounds ago, you have no option but to retrain a model with the appropriate number of iterations.
#             https://www.kaggle.com/c/microsoft-malware-prediction/discussion/78253
            if boosting_type=='dart':
        
                evaluation_score = lgbm.evals_result_

                dart_best_iteration = evaluation_score['valid_0']['l2'].index(min(evaluation_score['valid_0']['l2']))+1
            
                
                lgbm=LGBMRegressor(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                           n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                                num_iteration = dart_best_iteration,
                                boosting_type='dart', 
                                subsample = best_params['subsample'],
                                min_child_samples = int(best_params['min_child_samples']),
                                colsample_bytree=best_params['colsample_bytree'],
                                reg_alpha = best_params['reg_alpha'],
                                reg_lambda = best_params['reg_lambda'],
                                       silent = True)
                lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2')

            # Predictions on the training, cv, leaderboard and test set
            y_pred_train = lgbm.predict(X_train_no_na)
            y_pred_all = lgbm.predict(x_leaderboard)
            cv_preds = lgbm.predict(X_CV_no_na)
            # Save predictions of best model for leaderboard submition
            pred_all_backgrounds = lgbm.predict(background_imputed_tot)

        else:
            # Run bayesian optimization
            params = optimize_gbm_classifier(X_train_no_na, y_train_no_na,boosting_type)
            best_params = params['params']  
            print(0)
            # Validation accuracy
            
            # train a new model using the best hyperparameters 

            lgbm=LGBMClassifier(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                       n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                        num_iteration = int(best_params['num_iteration']),
                        boosting_type=boosting_type,
                        subsample = best_params['subsample'],
                        min_child_samples = int(best_params['min_child_samples']),
                        colsample_bytree=best_params['colsample_bytree'],
                        reg_alpha = best_params['reg_alpha'],
                        reg_lambda = best_params['reg_lambda'],   
                        silent = True)
    #         lgbm.fit(X_train_no_na, y_train_no_na,eval_metric='l1')
            lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2',
                 early_stopping_rounds = 10)
        
            if boosting_type=='dart':
                
                evaluation_score = lgbm.evals_result_

                dart_best_iteration = evaluation_score['valid_0']['l2'].index(min(evaluation_score['valid_0']['l2']))+1

                lgbm=LGBMClassifier(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                           n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                                num_iteration = dart_best_iteration,
                                boosting_type='dart', 
                                subsample = best_params['subsample'],
                                min_child_samples = int(best_params['min_child_samples']),
                                colsample_bytree=best_params['colsample_bytree'],
                                reg_alpha = best_params['reg_alpha'],
                                reg_lambda = best_params['reg_lambda'],
                                       silent = True)
                lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2')

            # Predictions on the training, cv, leaderboard and test set
            y_pred_train = lgbm.predict_proba(X_train_no_na)[:,1]
            y_pred_all = lgbm.predict_proba(x_leaderboard)[:,1]
            cv_preds = lgbm.predict_proba(X_CV_no_na)[:,1]
            # Save predictions of best model for leaderboard submition
            pred_all_backgrounds = lgbm.predict_proba(background_imputed_tot)[:,1]

        best_params_lgbm[variable] = best_params
        
        #### Saving model to be interpretted later
        joblib.dump(lgbm, 'lgbm_'+ variable + '_' + boosting_type + '.pkl')

        # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.       
        error_types = dict()  
        print('---- Train error ----')
        error_types['train'] = mean_squared_error(y_train_no_na,y_pred_train)
        print(mean_squared_error(y_train_no_na,y_pred_train))
        print('---- CV error ----')
        error_types['CV'] = mean_squared_error(y_CV_no_na,cv_preds)
        print(mean_squared_error(y_CV_no_na,cv_preds))

        print('---- leaderboard stats ----')
        print('r2 =',r2_score(y_leaderboard[~mask],y_pred_all[~mask]))
        
        error_types['leaderboard'] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
        print('mse =',mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
        
        lgbm_accuracies[variable] = error_types

        # lgbm=LGBMClassifier(learning_rate = 0.05, n_estimators = len(background.columns), num_leaves = 32)


        # Save predictions of best model for leaderboard submition

        prediction[variable] = pred_all_backgrounds

        lgbm_timings[variable] = datetime.now()-start
        
    best_params_each_model[boosting_type] = best_params_lgbm
    accuracies_each_model[boosting_type] = lgbm_accuracies
    timings_each_model[boosting_type] = lgbm_timings
    #     print(pred_all_backgrounds==cv_preds)


    prediction.to_csv('lgbm_preds_' + boosting_type + '.csv')
    print(datetime.now()-start)
    print('\n\n\n------------------------------------------')
best_params_each_model_no_sampling_copy = best_params_each_model.copy()


accuracies_each_model_no_sampling_copy = accuracies_each_model.copy()
    #     print(pred_all_backgrounds==cv_preds)

-------------  gbdt  --------------


gpa
gpa
|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_it... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
[-0.3847114  -0.39860344 -0.3823323  -0.36396924 -0.40016859 -0.40008174]
|  1        | -0.3883   |  0.9648   |  0.07011  |  3.575    |  48.87    |  572.0    |  139.3    |  29.01    |  0.6532   |  0.7489   |  0.8268   |
[-0.39797068 -0.39114754 -0.38533161 -0.37535318 -0.39039006 -0.41225703]
|  2        | -0.3921   |  0.8739   |  0.1926   |  1.117    |  30.22    |  305.7    |  148.5    |  24.67    |  0.8722   |  0.9646   |  0.8618   |
[-0.40053479 -0.41253653 -0.37121799 -0.38323532 -0.42760679 -0.43307693]
|  3        | -0.4047   |  0.8212   |  0.1463   |  7.546    |  71.86    |  445.2    |  159.5    |  29.83    |  0.6769   |  0.7908   |  0.5855   |
[-

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.419105
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.415402
[3]	valid_0's l2: 0.408967
[4]	valid_0's l2: 0.404659
[5]	valid_0's l2: 0.402966
[6]	valid_0's l2: 0.401038
[7]	valid_0's l2: 0.398723
[8]	valid_0's l2: 0.39693
[9]	valid_0's l2: 0.394334
[10]	valid_0's l2: 0.392788
[11]	valid_0's l2: 0.391387
[12]	valid_0's l2: 0.392798
[13]	valid_0's l2: 0.391322
[14]	valid_0's l2: 0.391016
[15]	valid_0's l2: 0.390686
[16]	valid_0's l2: 0.390656
[17]	valid_0's l2: 0.388981
[18]	valid_0's l2: 0.388777
[19]	valid_0's l2: 0.387407
[20]	valid_0's l2: 0.38591
[21]	valid_0's l2: 0.386525
[22]	valid_0's l2: 0.386503
[23]	valid_0's l2: 0.384589
[24]	valid_0's l2: 0.382543
[25]	valid_0's l2: 0.383436
[26]	valid_0's l2: 0.382759
[27]	valid_0's l2: 0.382086
[28]	valid_0's l2: 0.382311
[29]	valid_0's l2: 0.38188
[30]	valid_0's l2: 0.381776
[31]	valid_0's l2: 0.38086
[32]	valid_0's l2: 0.378829
[33]	valid_0's l2: 0.378894
[34]	valid_0's l2: 0.379418
[

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.212476
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 0.211777
[3]	valid_0's l2: 0.211368
[4]	valid_0's l2: 0.211052
[5]	valid_0's l2: 0.210399
[6]	valid_0's l2: 0.209799
[7]	valid_0's l2: 0.209816
[8]	valid_0's l2: 0.209972
[9]	valid_0's l2: 0.209968
[10]	valid_0's l2: 0.210099
[11]	valid_0's l2: 0.209387
[12]	valid_0's l2: 0.209259
[13]	valid_0's l2: 0.208825
[14]	valid_0's l2: 0.208828
[15]	valid_0's l2: 0.208333
[16]	valid_0's l2: 0.208057
[17]	valid_0's l2: 0.207857
[18]	valid_0's l2: 0.207707
[19]	valid_0's l2: 0.20791
[20]	valid_0's l2: 0.207568
[21]	valid_0's l2: 0.2075
[22]	valid_0's l2: 0.206995
[23]	valid_0's l2: 0.207
[24]	valid_0's l2: 0.206607
[25]	valid_0's l2: 0.206815
[26]	valid_0's l2: 0.20697
[27]	valid_0's l2: 0.207017
[28]	valid_0's l2: 0.206559
[29]	valid_0's l2: 0.206209
[30]	valid_0's l2: 0.205807
[31]	valid_0's l2: 0.205823
[32]	valid_0's l2: 0.205571
[33]	valid_0's l2: 0.205346
[34]	valid_0's l2: 0.205418
[35]

...

[1]	valid_0's l2: 0.048698	valid_0's binary_logloss: 0.199072
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.0482359	valid_0's binary_logloss: 0.195694
[3]	valid_0's l2: 0.0477263	valid_0's binary_logloss: 0.192095
[4]	valid_0's l2: 0.0474943	valid_0's binary_logloss: 0.190785
[5]	valid_0's l2: 0.0476496	valid_0's binary_logloss: 0.19193
[6]	valid_0's l2: 0.0472762	valid_0's binary_logloss: 0.189278
[7]	valid_0's l2: 0.0470248	valid_0's binary_logloss: 0.1874
[8]	valid_0's l2: 0.046945	valid_0's binary_logloss: 0.187246
[9]	valid_0's l2: 0.0470603	valid_0's binary_logloss: 0.188183
[10]	valid_0's l2: 0.0470186	valid_0's binary_logloss: 0.187988
[11]	valid_0's l2: 0.0470139	valid_0's binary_logloss: 0.187959
[12]	valid_0's l2: 0.0469724	valid_0's binary_logloss: 0.187344
[13]	valid_0's l2: 0.0470165	valid_0's binary_logloss: 0.187813
[14]	valid_0's l2: 0.0470072	valid_0's binary_logloss: 0.18796
[15]	valid_0's l2: 0.0470692	valid_0's binary_logloss: 0.1

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.154706	valid_0's binary_logloss: 0.488553
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.155438	valid_0's binary_logloss: 0.490573
[3]	valid_0's l2: 0.154705	valid_0's binary_logloss: 0.488596
[4]	valid_0's l2: 0.154686	valid_0's binary_logloss: 0.488574
[5]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[6]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[7]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[8]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[9]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[10]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[11]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[12]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[13]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[14]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
[15]	valid_0's l2: 0.154445	valid_0's binary_logloss: 0.487727
Ear

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.174001	valid_0's binary_logloss: 0.53216
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.17377	valid_0's binary_logloss: 0.531526
[3]	valid_0's l2: 0.173266	valid_0's binary_logloss: 0.530127
[4]	valid_0's l2: 0.172887	valid_0's binary_logloss: 0.529076
[5]	valid_0's l2: 0.172761	valid_0's binary_logloss: 0.528722
[6]	valid_0's l2: 0.172388	valid_0's binary_logloss: 0.527694
[7]	valid_0's l2: 0.171874	valid_0's binary_logloss: 0.526273
[8]	valid_0's l2: 0.171486	valid_0's binary_logloss: 0.525184
[9]	valid_0's l2: 0.171311	valid_0's binary_logloss: 0.524687
[10]	valid_0's l2: 0.171071	valid_0's binary_logloss: 0.524022
[11]	valid_0's l2: 0.170913	valid_0's binary_logloss: 0.523585
[12]	valid_0's l2: 0.170475	valid_0's binary_logloss: 0.522405
[13]	valid_0's l2: 0.170343	valid_0's binary_logloss: 0.522049
[14]	valid_0's l2: 0.170247	valid_0's binary_logloss: 0.521744
[15]	valid_0's l2: 0.170196	valid_0's binary_logloss: 0.521611
[16]	

In [6]:
# Save model performance and parameters
import pickle

# Save models and params
with open('best_params_tress_no_sampling.pickle', 'wb') as handle:
    pickle.dump(best_params_each_model_no_sampling_copy, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('best_params_tress_no_sampling.pickle', 'wb') as handle:
    pickle.dump(accuracies_each_model_no_sampling_copy, handle, protocol=pickle.HIGHEST_PROTOCOL)

# best_params_each_model_no_sampling_copy
# accuracies_each_model_no_sampling_copy

In [7]:
# Print accuracies
lst_leader_board_accs = list()
for model_name in accuracies_each_model:
    for var_name in accuracies_each_model[model_name]:
        lst_leader_board_accs.append(var_name + ' ' + model_name + ' ' +  str(accuracies_each_model[model_name][var_name]['leaderboard']))

sorted(lst_leader_board_accs)

['eviction dart 0.05207330412177481',
 'eviction gbdt 0.050633443020071826',
 'eviction goss 0.05192825850648769',
 'gpa dart 0.38124499706416043',
 'gpa gbdt 0.37764382969532606',
 'gpa goss 0.3710648078690564',
 'grit dart 0.2169031608664567',
 'grit gbdt 0.21711522212374287',
 'grit goss 0.21659011055012237',
 'jobTraining dart 0.20428263080049058',
 'jobTraining gbdt 0.19940966236005536',
 'jobTraining goss 0.2032936824781467',
 'layoff dart 0.1744041463446064',
 'layoff gbdt 0.17419679113048317',
 'layoff goss 0.17438181236965813',
 'materialHardship dart 0.0255595316615577',
 'materialHardship gbdt 0.02609444653819857',
 'materialHardship goss 0.026143124475680617']

### Running the exact same code as the non-weighted model, except using balanced class weights to increase the weights associated with the minority class. Model calibration is used after training the models, since the probability become biased towards the minority class. This code only trains models for the categorical outcomes, since continuous outcomes do not need to be balanced

In [9]:
from datetime import datetime 
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV
from datetime import datetime
from sklearn.model_selection import RepeatedKFold,RepeatedStratifiedKFold
from sklearn.calibration import CalibratedClassifierCV


variables = test.columns.to_list()


prediction = pd.read_csv('FFChallenge_v5/prediction.csv', sep=',', header=0, index_col=0)



def optimize_gbm_regressor(data, targets,boosting_type):
    """Apply Bayesian Optimization to GBM parameters."""

    def gbm_crossval(num_leaves, max_depth, learning_rate,
                     n_estimators,num_iteration,subsample,min_child_samples,
                    colsample_bytree,reg_alpha,reg_lambda,):
        """GB cross validation.
        """
        clf = LGBMRegressor(
            num_leaves=int(num_leaves),
            max_depth=int(max_depth),
            learning_rate=learning_rate,
            n_estimators=int(n_estimators),
            num_iteration=int(num_iteration),
            boosting_type = boosting_type,
            subsample = subsample,
            min_child_samples = int(min_child_samples),
            colsample_bytree=colsample_bytree,
            reg_alpha = reg_alpha,
            reg_lambda = reg_lambda,
        )
        cv = RepeatedKFold(n_splits=3, n_repeats=2, random_state=1)
        cval = cross_val_score(clf, data, targets, scoring='neg_mean_squared_error', cv=cv,n_jobs=-1)
        print(cval)
        return cval.mean()
    
    # Set up the BO
    optimizer = BayesianOptimization(
        f=gbm_crossval,
        pbounds={
            'num_leaves': (2, 30),
            'max_depth': (1, 15),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (10,1000),
            'num_iteration':(50,200),
            'subsample': (0.5,1.0),
            'min_child_samples' : (10, 200),
            'colsample_bytree':(0.5,1),
            'reg_alpha': (0.0, 1.0),
            'reg_lambda': (0.0, 1.0),
        },
        random_state=12345,
        verbose=3
    )
    optimizer.maximize(n_iter=15, init_points=5)

    print("Final result:", optimizer.max)
    return optimizer.max



def optimize_gbm_classifier(data, targets,boosting_type):
    """Apply Bayesian Optimization to GBM parameters."""
#     self.boosting_type = boosting_type

    def gbm_crossval(num_leaves, max_depth, learning_rate,
                     n_estimators,num_iteration,subsample,min_child_samples,
                    colsample_bytree,reg_alpha,reg_lambda,):
        """GB cross validation.
        """
        clf = LGBMClassifier(
        num_leaves=int(num_leaves),
        max_depth=int(max_depth),
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        num_iteration=int(num_iteration),
        boosting_type = boosting_type,
        subsample = subsample,
        min_child_samples = int(min_child_samples),
        colsample_bytree=colsample_bytree,
        reg_alpha = reg_alpha,
        reg_lambda = reg_lambda,
        class_weight = 'balanced',
        )
        cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=123)
        cval = cross_val_score(clf, data, targets, scoring='neg_brier_score', cv=cv,n_jobs=-1)
#         print(cval)
        return cval.mean()
    
    # Set up the BO
    optimizer = BayesianOptimization(
        f=gbm_crossval,
        pbounds={
            'num_leaves': (2, 30),
            'max_depth': (1, 15),
            'learning_rate': (0.01, 0.2),
            'n_estimators': (10,1000),
            'num_iteration':(50,200),
            'subsample': (0.5,1.0),
            'min_child_samples' : (10, 200),
            'colsample_bytree':(0.5,1),
            'reg_alpha': (0.0, 1.0),
            'reg_lambda': (0.0, 1.0),
        },
        random_state=12345,
#         verbose=1
    )
    optimizer.maximize(n_iter=15, init_points=5)

    print("Final result:", optimizer.max)
    return optimizer.max


classification_vars = ['eviction','layoff','jobTraining']




boosting_types = ['gbdt','dart','goss']

best_params_each_model = dict()

accuracies_each_model = dict()
for boosting_type in boosting_types:
    print('------------- ',boosting_type, ' --------------')
    print('\n')
    lgbm_accuracies = dict()
    for variable in variables[3:]: # Run for the three categorical outcomes
#     for variable in ['grit','layoff']:
    # variable = 'eviction'
        print(variable)
        start=datetime.now()
        # Let's drop instances where the response is NA
        y_train_no_na = y_train[variable].dropna()
        X_train_no_na = X_train.loc[y_train_no_na.index.values]

        y_CV_no_na = y_CV[variable].dropna()
        X_CV_no_na = X_CV.loc[y_CV_no_na.index.values]

        mask = leaderboard[variable].isna()
        y_leaderboard = leaderboard[variable]

        ###### model = Lasso(alpha=1000.0,max_iter=10000)
        # model.fit(X_train, y_train_trans)
        # from imblearn.over_sampling import SMOTE

        # train_data=lgb.Dataset(X_train,label=y_train_trans)
        #setting parameters for lightgbm
        # param = {'num_leaves':50, 'objective':'binary','max_depth':7,'learning_rate':.2,'max_bin':200}
        # param['metric'] = ['auc', 'binary_logloss']

        #training our model using light gbm

        # specify parameters and distributions to sample from

        if variable not in classification_vars:
            print(variable)
            params = optimize_gbm_regressor(X_train_no_na, y_train_no_na,boosting_type)
            best_params = params['params']  
            print(1)

            # Validation accuracy 
            lgbm=LGBMRegressor(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                   n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                        num_iteration = int(best_params['num_iteration']),
                        boosting_type=boosting_type, 
                        subsample = best_params['subsample'],
                        min_child_samples = int(best_params['min_child_samples']),
                        colsample_bytree=best_params['colsample_bytree'],
                        reg_alpha = best_params['reg_alpha'],
                        reg_lambda = best_params['reg_lambda'],
                               silent = True)

            # I used early stopping to help prevent overfitting the model on the training set (stops the model once it's performance doesn't
            # improve for 5 iterations on the validation set)
            lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2',
             early_stopping_rounds = 5)

            
            # For early stopping (i.e. early stopping at 10 rounds with no improvement to cv)
            # The model that displayed the score 10 rounds ago does not exist anymore (a few trees have been dropped).
            # To get the best score you got 20 rounds ago, you have no option but to retrain a model with the appropriate number of iterations.
#             https://www.kaggle.com/c/microsoft-malware-prediction/discussion/78253
            if boosting_type=='dart':
        
                evaluation_score = lgbm.evals_result_

                dart_best_iteration = evaluation_score['valid_0']['l2'].index(min(evaluation_score['valid_0']['l2']))

                lgbm=LGBMRegressor(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                           n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                                num_iteration = dart_best_iteration,
                                boosting_type='dart', 
                                subsample = best_params['subsample'],
                                min_child_samples = int(best_params['min_child_samples']),
                                colsample_bytree=best_params['colsample_bytree'],
                                reg_alpha = best_params['reg_alpha'],
                                reg_lambda = best_params['reg_lambda'],
                                       silent = True)
                lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2')


            y_pred_train = lgbm.predict(X_train_no_na)
            y_pred_all = lgbm.predict(x_leaderboard)
            cv_preds = lgbm.predict(X_CV_no_na)
            # Save predictions of best model for leaderboard submition
            pred_all_backgrounds = lgbm.predict(background_imputed_tot)

        else:
            #         'gbdt', 'goss', 'dart', 'rf'
            params = optimize_gbm_classifier(X_train_no_na, y_train_no_na,boosting_type)
            best_params = params['params']  
            print(0)
            # Validation accuracy

            lgbm=LGBMClassifier(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                       n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                        num_iteration = int(best_params['num_iteration']),
                        boosting_type=boosting_type,
                        subsample = best_params['subsample'],
                        min_child_samples = int(best_params['min_child_samples']),
                        colsample_bytree=best_params['colsample_bytree'],
                        reg_alpha = best_params['reg_alpha'],
                        reg_lambda = best_params['reg_lambda'],
                        class_weight = 'balanced',
                        silent = True)
    #         lgbm.fit(X_train_no_na, y_train_no_na,eval_metric='l1')
            lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2',
                 early_stopping_rounds = 10)
        
            if boosting_type=='dart':
                
                evaluation_score = lgbm.evals_result_

                dart_best_iteration = evaluation_score['valid_0']['l2'].index(min(evaluation_score['valid_0']['l2'])) + 1

                lgbm=LGBMClassifier(learning_rate = best_params['learning_rate'], max_depth =  int(best_params['max_depth']),
                           n_estimators = int(best_params['n_estimators']), num_leaves = int(best_params['num_leaves']),
                                num_iteration = dart_best_iteration,
                                boosting_type='dart', 
                                subsample = best_params['subsample'],
                                min_child_samples = int(best_params['min_child_samples']),
                                colsample_bytree=best_params['colsample_bytree'],
                                reg_alpha = best_params['reg_alpha'],
                                reg_lambda = best_params['reg_lambda'],
                                class_weight = 'balanced',
                                       silent = True)
                lgbm.fit(X_train_no_na, y_train_no_na,eval_set = (X_CV_no_na,y_CV_no_na), eval_metric='l2')
            
            
            # Calibrate after training the model on the best parameters using a sigmoid calibration function
            calibrated = CalibratedClassifierCV(lgbm, cv='prefit', method='sigmoid')
            calibrated.fit(X_CV_no_na, y_CV_no_na)

            y_pred_train = calibrated.predict_proba(X_train_no_na)[:,1]
            y_pred_all = calibrated.predict_proba(x_leaderboard)[:,1]
            cv_preds = calibrated.predict_proba(X_CV_no_na)[:,1]
                        # Save predictions of best model for leaderboard submition
            pred_all_backgrounds =calibrated.predict_proba(background_imputed_tot)[:,1]


#             y_pred_train = lgbm.predict_proba(X_train_no_na)[:,1]
#             y_pred_all = lgbm.predict_proba(x_leaderboard)[:,1]
#             cv_preds = lgbm.predict_proba(X_CV_no_na)[:,1]
#             # Save predictions of best model for leaderboard submition
#             pred_all_backgrounds = lgbm.predict_proba(background_imputed_tot)[:,1]

        best_params_lgbm[variable] = best_params


        # Can use mean square error as performance metric for both, since for 1 output, the mean square error and brier loss are the same.
        
        error_types = dict()  
        print('---- Train error ----')
        error_types['train'] = mean_squared_error(y_train_no_na,y_pred_train)
        print(mean_squared_error(y_train_no_na,y_pred_train))
        print('---- CV error ----')
        error_types['CV'] = mean_squared_error(y_CV_no_na,cv_preds)
        print(mean_squared_error(y_CV_no_na,cv_preds))

        print('---- leaderboard stats ----')
        print(r2_score(y_leaderboard[~mask],y_pred_all[~mask]))
        
        error_types['leaderboard'] = mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask])
        print(mean_squared_error(y_leaderboard[~mask],y_pred_all[~mask]))
        
        lgbm_accuracies[variable] = error_types

        # lgbm=LGBMClassifier(learning_rate = 0.05, n_estimators = len(background.columns), num_leaves = 32)


        # Save predictions of best model for leaderboard submition

        prediction[variable] = pred_all_backgrounds


    best_params_each_model[boosting_type] = best_params_lgbm
    accuracies_each_model[boosting_type] = lgbm_accuracies
    #     print(pred_all_backgrounds==cv_preds)


    prediction.to_csv('lgbm_preds_' + boosting_type + '.csv')
    print(datetime.now()-start)
    print('\n\n\n------------------------------------------')
best_params_each_model_weighted_copy = best_params_each_model.copy()

-------------  gbdt  --------------


eviction
|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_it... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
|  1        | -0.05957  |  0.9648   |  0.07011  |  3.575    |  48.87    |  572.0    |  139.3    |  29.01    |  0.6532   |  0.7489   |  0.8268   |
|  2        | -0.0814   |  0.8739   |  0.1926   |  1.117    |  30.22    |  305.7    |  148.5    |  24.67    |  0.8722   |  0.9646   |  0.8618   |
|  3        | -0.05879  |  0.8212   |  0.1463   |  7.546    |  71.86    |  445.2    |  159.5    |  29.83    |  0.6769   |  0.7908   |  0.5855   |
|  4        | -0.06     |  0.5134   |  0.1621   |  13.65    |  14.69    |  496.8    |  128.9    |  18.7     |  0.05196  |  0.8951   |  0.8641   |
|  5        | -0.05937  |  0.9092   |  0.105    |  12.34    |  28.23    |  22

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.196833	valid_0's binary_logloss: 0.585557
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.159369	valid_0's binary_logloss: 0.505307
[3]	valid_0's l2: 0.132265	valid_0's binary_logloss: 0.443251
[4]	valid_0's l2: 0.110814	valid_0's binary_logloss: 0.391198
[5]	valid_0's l2: 0.0977501	valid_0's binary_logloss: 0.355265
[6]	valid_0's l2: 0.0860269	valid_0's binary_logloss: 0.321635
[7]	valid_0's l2: 0.0801892	valid_0's binary_logloss: 0.301981
[8]	valid_0's l2: 0.0743954	valid_0's binary_logloss: 0.282345
[9]	valid_0's l2: 0.068289	valid_0's binary_logloss: 0.263569
[10]	valid_0's l2: 0.0635337	valid_0's binary_logloss: 0.248025
[11]	valid_0's l2: 0.0612069	valid_0's binary_logloss: 0.238546
[12]	valid_0's l2: 0.0597573	valid_0's binary_logloss: 0.231027
[13]	valid_0's l2: 0.0584132	valid_0's binary_logloss: 0.223961
[14]	valid_0's l2: 0.056505	valid_0's binary_logloss: 0.216696
[15]	valid_0's l2: 0.0543107	valid_0's binary_logloss: 0.2

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.246323	valid_0's binary_logloss: 0.685778
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.242177	valid_0's binary_logloss: 0.677442
[3]	valid_0's l2: 0.23848	valid_0's binary_logloss: 0.669953
[4]	valid_0's l2: 0.238591	valid_0's binary_logloss: 0.670141
[5]	valid_0's l2: 0.235823	valid_0's binary_logloss: 0.664422
[6]	valid_0's l2: 0.23212	valid_0's binary_logloss: 0.656786
[7]	valid_0's l2: 0.231657	valid_0's binary_logloss: 0.655802
[8]	valid_0's l2: 0.229908	valid_0's binary_logloss: 0.652061
[9]	valid_0's l2: 0.229266	valid_0's binary_logloss: 0.65065
[10]	valid_0's l2: 0.227135	valid_0's binary_logloss: 0.646194
[11]	valid_0's l2: 0.225432	valid_0's binary_logloss: 0.642769
[12]	valid_0's l2: 0.221741	valid_0's binary_logloss: 0.634908
[13]	valid_0's l2: 0.216964	valid_0's binary_logloss: 0.624762
[14]	valid_0's l2: 0.214077	valid_0's binary_logloss: 0.618468
[15]	valid_0's l2: 0.214203	valid_0's binary_logloss: 0.618388
[16]	v

...

[1]	valid_0's l2: 0.238991	valid_0's binary_logloss: 0.671065
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.227768	valid_0's binary_logloss: 0.648318
[3]	valid_0's l2: 0.218474	valid_0's binary_logloss: 0.629025
[4]	valid_0's l2: 0.209854	valid_0's binary_logloss: 0.610995
[5]	valid_0's l2: 0.205295	valid_0's binary_logloss: 0.601279
[6]	valid_0's l2: 0.200619	valid_0's binary_logloss: 0.591214
[7]	valid_0's l2: 0.194947	valid_0's binary_logloss: 0.579028
[8]	valid_0's l2: 0.186722	valid_0's binary_logloss: 0.561079
[9]	valid_0's l2: 0.179853	valid_0's binary_logloss: 0.54572
[10]	valid_0's l2: 0.176105	valid_0's binary_logloss: 0.536974
[11]	valid_0's l2: 0.172306	valid_0's binary_logloss: 0.528337
[12]	valid_0's l2: 0.171163	valid_0's binary_logloss: 0.525414
[13]	valid_0's l2: 0.167764	valid_0's binary_logloss: 0.517788
[14]	valid_0's l2: 0.166181	valid_0's binary_logloss: 0.513851
[15]	valid_0's l2: 0.165007	valid_0's binary_logloss: 0.510892
[16]

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.247972	valid_0's binary_logloss: 0.689062
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.240837	valid_0's binary_logloss: 0.674676
[3]	valid_0's l2: 0.2399	valid_0's binary_logloss: 0.672653
[4]	valid_0's l2: 0.236051	valid_0's binary_logloss: 0.664889
[5]	valid_0's l2: 0.231613	valid_0's binary_logloss: 0.655764
[6]	valid_0's l2: 0.227722	valid_0's binary_logloss: 0.647446
[7]	valid_0's l2: 0.223672	valid_0's binary_logloss: 0.639138
[8]	valid_0's l2: 0.222018	valid_0's binary_logloss: 0.635341
[9]	valid_0's l2: 0.221701	valid_0's binary_logloss: 0.634902
[10]	valid_0's l2: 0.227426	valid_0's binary_logloss: 0.646666
[11]	valid_0's l2: 0.234005	valid_0's binary_logloss: 0.660216
[12]	valid_0's l2: 0.237167	valid_0's binary_logloss: 0.666965
[13]	valid_0's l2: 0.241224	valid_0's binary_logloss: 0.675662
[14]	valid_0's l2: 0.246094	valid_0's binary_logloss: 0.68642
[15]	valid_0's l2: 0.250488	valid_0's binary_logloss: 0.696419
[16]	v

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l2: 0.245308	valid_0's binary_logloss: 0.683734
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l2: 0.238096	valid_0's binary_logloss: 0.669196
[3]	valid_0's l2: 0.231505	valid_0's binary_logloss: 0.655743
[4]	valid_0's l2: 0.226697	valid_0's binary_logloss: 0.645817
[5]	valid_0's l2: 0.223452	valid_0's binary_logloss: 0.638997
[6]	valid_0's l2: 0.219753	valid_0's binary_logloss: 0.631204
[7]	valid_0's l2: 0.215409	valid_0's binary_logloss: 0.622165
[8]	valid_0's l2: 0.213482	valid_0's binary_logloss: 0.618025
[9]	valid_0's l2: 0.211757	valid_0's binary_logloss: 0.61416
[10]	valid_0's l2: 0.21688	valid_0's binary_logloss: 0.624879
[11]	valid_0's l2: 0.218807	valid_0's binary_logloss: 0.628717
[12]	valid_0's l2: 0.22171	valid_0's binary_logloss: 0.634949
[13]	valid_0's l2: 0.223636	valid_0's binary_logloss: 0.638757
[14]	valid_0's l2: 0.22591	valid_0's binary_logloss: 0.64348
[15]	valid_0's l2: 0.223654	valid_0's binary_logloss: 0.638227
[16]	val

In [10]:
import pickle

# Save models and params and accuracies
with open('best_params_trees_weighted.pickle', 'wb') as handle:
    pickle.dump(best_params_each_model_weighted_copy, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('best_accs_trees_weighted.pickle', 'wb') as handle:
    pickle.dump(accuracies_each_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

# best_params_each_model_no_sampling_copy
# accuracies_each_model_no_sampling_copy

In [11]:

lst_leader_board_accs = list()
for model_name in accuracies_each_model:
    for var_name in accuracies_each_model[model_name]:
        lst_leader_board_accs.append(var_name + ' ' + model_name + ' ' +  str(accuracies_each_model[model_name][var_name]['leaderboard']))

sorted(lst_leader_board_accs)

['eviction dart 0.05315720683920307',
 'eviction gbdt 0.05256860701916669',
 'eviction goss 0.052061264545674536',
 'jobTraining dart 0.2118030106777266',
 'jobTraining gbdt 0.21187664440564227',
 'jobTraining goss 0.20413801623096692',
 'layoff dart 0.17664021831573956',
 'layoff gbdt 0.17505472493917246',
 'layoff goss 0.17617282798869383']